### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from aws_setup import *

In [4]:
name='fast-ai'

In [5]:
vpc = create_vpc(name)

In [9]:
list(ec2.vpcs.filter(Filters=[{'Name':'tag:Name', 'Values':[name]}]))

[ec2.Vpc(id='vpc-ee793297')]

In [11]:
create_ec2_keypair(name)

Created keypair


#### Request Spot instance

In [25]:
instance_name = f'{name}-instance'
instance_type = 'c5.xlarge'

In [26]:
spot_prices = get_spot_prices(); spot_prices[instance_type]

'0.062500'

In [21]:
sorted(spot_prices.items(), key=lambda x: float(x[1]))

[('m3.medium', '0.006700'),
 ('m1.medium', '0.008700'),
 ('t2.medium', '0.013900'),
 ('c3.large', '0.027300'),
 ('c4.large', '0.028700'),
 ('r3.large', '0.030800'),
 ('m2.2xlarge', '0.049000'),
 ('c1.xlarge', '0.052000'),
 ('t2.xlarge', '0.055700'),
 ('t2.large', '0.055800'),
 ('m3.xlarge', '0.057400'),
 ('c4.xlarge', '0.059300'),
 ('c3.xlarge', '0.059800'),
 ('m4.xlarge', '0.060200'),
 ('c5.xlarge', '0.062500'),
 ('r3.xlarge', '0.069400'),
 ('r4.xlarge', '0.085600'),
 ('m2.4xlarge', '0.098000'),
 ('i3.xlarge', '0.102600'),
 ('c3.2xlarge', '0.113000'),
 ('c5.large', '0.122100'),
 ('r4.large', '0.123500'),
 ('m5.large', '0.123600'),
 ('m4.2xlarge', '0.123600'),
 ('r3.2xlarge', '0.133300'),
 ('c5.2xlarge', '0.140600'),
 ('m5.2xlarge', '0.147600'),
 ('r4.2xlarge', '0.154700'),
 ('c4.2xlarge', '0.160700'),
 ('h1.2xlarge', '0.165000'),
 ('m1.large', '0.175000'),
 ('i3.2xlarge', '0.191800'),
 ('m1.xlarge', '0.219000'),
 ('c3.4xlarge', '0.235000'),
 ('c4.4xlarge', '0.241100'),
 ('g2.2xlarge',

In [24]:
sorted([o for o in spot_prices.items() if o[0].startswith('c5')], key=lambda x: float(x[1]))

[('c5.xlarge', '0.062500'),
 ('c5.large', '0.122100'),
 ('c5.2xlarge', '0.140600'),
 ('c5.4xlarge', '0.304000'),
 ('c5.18xlarge', '1.150100'),
 ('c5.9xlarge', '2.197600')]

In [27]:
launch_specs = LaunchSpecs(vpc, instance_type=instance_type).build()

In [28]:
instance = create_spot_instance(instance_name, launch_specs); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-00e902bf70c696999
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@35.162.88.124


ec2.Instance(id='i-00e902bf70c696999')

Request on demand instance (if spot error)

In [68]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [29]:
instance = get_instance(f'{instance_name}'); instance

ec2.Instance(id='i-00e902bf70c696999')

#### Attach EBS volume (Optional)

In [30]:
volume_tag = f'{name}-ebs-volume'

In [40]:
az = list(vpc.network_interfaces.all())[0].availability_zone
az

'us-west-2b'

In [42]:
az

'us-west-2b'

In [43]:
volume = create_volume(volume_tag, az=az, size=100)

ClientError: An error occurred (InvalidParameterValue) when calling the CreateVolume operation: 'null' is not a valid taggable resource type for this operation.

In [ ]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

#### Create EFS

In [45]:
efs_tag = f'{name}-efs'

In [47]:
efs = create_efs(efs_tag, vpc)

### SSH into instance

In [48]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EBS

In [49]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

('', '')

In [50]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

In [51]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

#### Mount EFS

In [52]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-080292a1.efs.us-west-2.amazonaws.com'

In [53]:
out, _ = run_command(client, 'mkdir ~/efs_mount')

In [54]:
mount_opts = 'nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2'
efs_mount_cmd = f'sudo mount -t nfs -o {mount_opts} {efs_addr}:/ ~/efs_mount'

In [59]:
out, _ = run_command(client, efs_mount_cmd)

In [60]:
out

''

In [61]:
client.close()

In [63]:
keypath=f'{Path.home()}/.ssh/aws-key-fast-ai.pem'
username='ubuntu'
f'ssh -i {keypath} {username}@{instance.public_ip_address}'

'ssh -i /home/jhoward/.ssh/aws-key-fast-ai.pem ubuntu@35.162.88.124'